In [1]:
import pandas as pd 
import numpy as np
import scipy
import time
from typing import Literal
from kee_utils import read_data_list

 ### **Combining the experimental outcome**

In [5]:
# ds = ['dermatology-6', 'ecoli-0-1-4-6_vs_5', 'glass-0-1-6_vs_5', 'glass-0-4_vs_5', 'glass-0-6_vs_5', 'newthyroid2',\
    # 'poker-8_vs_6', 'poker-9_vs_7', 'shuttle-6_vs_2-3', 'shuttle-c0-vs-c4', 'shuttle-c2-vs-c4', 'vowel0',\
    # 'winequality-red-3_vs_5','winequality-white-3-9_vs_5', 'yeast-0-2-5-7-9_vs_3-6-8','yeast-1_vs_7','yeast-2_vs_8']

def Combining_experimetal_result(
        input_path : str,
        classifier_list : list,
        test_method : str ,
        metrics_list : list,
        ):
    outcome = {}
    for clf in classifier_list:
        path = input_path + test_method + '_' + clf +'.csv'
        # path = './Experimental_result/' + 'KM5/CIBO_' + clf + '_KM5' +'.csv'
        data = pd.read_csv(path, header = 0)
        outcome[clf] = {}
        outcome[clf]['AUC'] = data['AUC']
        outcome[clf]['F1-score'] = data['F1-score']
        outcome[clf]['G-mean'] = data['G-mean']
    # print(RDBU_outcome['MLP']['F1-score'])

    temp = {}
    pic = []
    for clf in classifier_list:
        temp[clf] = {}
        for metric in metrics_list:
            path = './Experimental_result/Comparision_6c_3m_1110/' + clf + '_Comparision.xlsx'
            data = pd.read_excel(path, engine='openpyxl', sheet_name = metric)

            # print(data.columns)
            # data.columns = ['Dataset', 'None', 'ROS', 'SMOTE', 'B-SMOTE', 'ADASYN', 'SL_SMOTE', 'MWMOTE', 'KMeansSMOTE', 'NRAS', 'SOMO', 'PF_SMOTE_ORG']
            data[test_method] = outcome[clf][metric]
            data.index = data['Unnamed: 0']
            data = data.iloc[:, 1:]
            temp[clf][metric] = data
            
            ###########################################################
            # select_id = [i for i in range(data.shape[0]) if data.iloc[i, 0] in ds]
            # selection of dataset
            # data = data.iloc[select_id, :].reset_index(drop = True)
            # data.to_excel('C:/Users/KEE/Desktop/eval_outcome/' + clf + '_' + metric, engine = 'openpyxl')
            ###########################################################
        # create file
        address = input_path + clf + '_Comparison.xlsx'
        temp2 = pd.DataFrame()
        temp2.to_excel(address, sheet_name = 'Initialization', index = True)
        writer = pd.ExcelWriter(address, mode = 'a')
        for metric in metrics_list:
            temp[clf][metric].to_excel(writer, sheet_name = metric, index = True, header = True)
        writer.close()
    return -1
# clf_list = ['DT', 'NB', 'RF',  'SVM']#, 'KNN','MLP'
# test_method = 'CIBO'
# input_path = './Experimental_result/KM5/'
# metrics_list = ['AUC', 'F1-score', 'G-mean']

# Combining_experimetal_result(input_path = input_path, test_method = test_method, classifier_list = clf_list, metrics_list = metrics_list)

### **Friedman test**

In [6]:

def Friedman_test(
        input_path : str,
        classifier_list : list,
        ):

    p_value = pd.DataFrame({}, columns = ['p_value'], index=[])
    for clf in classifier_list:
        excel = pd.ExcelFile(input_path + clf + '_Comparison.xlsx')
        metrics_list = excel.sheet_names[1:]
        for metric in metrics_list:
            
            data = pd.read_excel(
                excel, 
                sheet_name = metric)
            
 
            p = scipy.stats.friedmanchisquare(*[data[x] for x in list(data.columns)[1:]])[1]
            p = pd.DataFrame([p], columns = ['p_value'], index=[clf + '_' + metric])
            p_value = pd.concat([p_value, p])

    p_value.to_excel(input_path + 'Friedman_Test.xlsx', sheet_name = 'FriedmanTest', index = True)
    # print(p_value)

    return -1
# clf_list = ['DT', 'NB', 'RF',  'SVM']#, 'KNN','MLP'
# test_method = 'CIBO'
# input_path = './Experimental_result/KM5/'
# metrics_list = ['AUC', 'F1-score', 'G-mean']
# Friedman_test(input_path = input_path, classifier_list = clf_list)


### **Wilcoxon signed-rank test**

In [7]:
# ds = ['dermatology-6', 'ecoli-0-1-4-6_vs_5', 'ecoli4', 'glass-0-1-4-6_vs_2','glass-0-1-6_vs_5', 'glass-0-4_vs_5', \
#     'glass-0-6_vs_5', 'new-thyroid1','newthyroid2', 'poker-8_vs_6', 'poker-9_vs_7', 'segment0', 'shuttle-2_vs_5',\
#     'shuttle-6_vs_2-3', 'shuttle-c0-vs-c4', 'shuttle-c2-vs-c4', 'winequality-red-3_vs_5', 'winequality-red-4', \
#     'winequality-red-8_vs_6', 'winequality-white-3-9_vs_5', 'yeast-0-2-5-7-9_vs_3-6-8', 'yeast-2_vs_4', 'yeast-2_vs_8', 'zoo-3']

def Wilcoxon_signed_rank_test(
        input_path : str,
        classifier_list : list,
        compared_method : str,
        ):
    

    temp1 = pd.DataFrame()
    temp1.to_excel(input_path + 'Wilcoxon_signed_rank_Test.xlsx', sheet_name = 'Initialization', index = True)
    writer = pd.ExcelWriter(input_path + 'Wilcoxon_signed_rank_Test.xlsx', mode = 'a')
    for clf in classifier_list:
        excel = pd.ExcelFile(input_path + clf + '_Comparison.xlsx')
        metrics_list = excel.sheet_names[1:]
        p_value = pd.DataFrame({})
        for metric in metrics_list:
            # Input experimental result metric by metric
            data = pd.read_excel(
                excel, 
                sheet_name = metric)

            comparison = ['CIBO' + " v.s "  + x  for x in data.columns[1:-1]]
            p_list = []


            for method in data.columns[1:-1]:
                p = scipy.stats.wilcoxon(x = data[compared_method], y = data[method], alternative = 'greater')[1]
                p_list.append(p)
            p_value[clf + '_' + metric] = p_list
        comparison = ['CIBO' + " v.s "  + x  for x in data.columns[1:-1]]
        p_value.index = comparison
        # print(p_value)        
        p_value.to_excel(writer, sheet_name = clf, index = True, header = True)
    writer.close()

    return -1
# clf_list = ['DT', 'NB', 'RF',  'SVM']#, 'KNN','MLP'
# test_method = 'CIBO'
# input_path = './Experimental_result/KM5/'

# Wilcoxon_signed_rank_test(input_path = input_path, classifier_list = clf_list, compared_method = test_method)

### **AGG Mean rank**

In [58]:
def rank_table(
        input_path : str,
        classifier_list : list,
        clustering_list : list,
        num_col : str,
        rank_method : Literal['average', 'min', 'max', 'first', 'dense'] = 'average'
        # output_path : str,
        ):
    
    outcome = { x : np.empty(shape = (1, num_col + 2)) for x in ['AUC', 'F1-score', 'G-mean']}

    for clus in clustering_list:
        for clf in classifier_list:
            excel = pd.ExcelFile(input_path + clus + '/' + clf + '_Comparison.xlsx')
            metrics_list = excel.sheet_names[1:]
            for metric in metrics_list:
                # Input experimental result metric by metric
                data = pd.read_excel(
                    excel, 
                    sheet_name = metric)
                rank_table = data.iloc[:, 1:].rank(axis = 1, ascending = False, method = rank_method)
                avg_rank = rank_table.mean(axis = 0)
                avg_rank = np.hstack((avg_rank, [clus, clf]))
                outcome[metric] = np.vstack((outcome[metric], avg_rank))

    col_name = list(data.columns[1:])
    col_name+=['Clustering', 'Classifier']
    temp1 = pd.DataFrame()
    temp1.to_excel(input_path + 'Rank_Table_new.xlsx' + rank_method, sheet_name = 'Initialization', index = True)
    writer = pd.ExcelWriter(input_path + 'Rank_Table_new.xlsx' + rank_method, mode = 'a')
    for m in ['AUC', 'F1-score', 'G-mean']:
        outcome[m] = outcome[m][1:, :]
        outcome[m] = pd.DataFrame(outcome[m], columns = col_name)
        outcome[m] = outcome[m][['Clustering', 'Classifier', 'None', 'ROS', 'SMOTE', 'B-SMOTE', 'ADASYN', 'SL_SMOTE', 'MWMOTE', 'KMeansSMOTE', 'NRAS', 'SOMO', 'PF_SMOTE_ORG', 'CIBO']]
        outcome[m]['Rank'] = outcome[m].iloc[:, 2:].rank(axis = 1).iloc[:,-1]

        outcome[m][['None', 'ROS', 'SMOTE', 'B-SMOTE', 'ADASYN', 'SL_SMOTE', 'MWMOTE', 'KMeansSMOTE', 'NRAS', 'SOMO', 'PF_SMOTE_ORG', 'CIBO']] = \
            outcome[m][['None', 'ROS', 'SMOTE', 'B-SMOTE', 'ADASYN', 'SL_SMOTE', 'MWMOTE', 'KMeansSMOTE', 'NRAS', 'SOMO', 'PF_SMOTE_ORG', 'CIBO']].astype('float')
        
        outcome[m].to_excel(writer, sheet_name = m, index = False, header = True)

    writer.close()
    return -1

clusterung_list = ['KM4', 'KM5', 'AHC4', 'AHC5', 'HDB_with_noise'] #
classifier_list = ['DT', 'RF', 'NB', 'MLP', 'SVM'] #
rank_table('./Experimental_result/Final_result/', classifier_list = classifier_list, clustering_list = clustering_list, num_col = 12)


-1

### **Decimal Seperation**

In [17]:
def decimal_seperation(x, truncate : int = 4):
    _ = '%e' % x
    num = round(float(_.partition('e')[0]), truncate)
    sci_sign = int(_.partition('e')[-1])

    if len(str(num)) - 2  != truncate:
        fill_zero = truncate - (len(str(num))- 2)
        return str(num) + '0'*fill_zero + 'x10^' + str(sci_sign)
    else:
        return str(num) + 'x10^' + str(sci_sign)

decimal_seperation(0.00000125011)

'1.2501x10^-6'

### **AGG P-value of Friedman test**

In [98]:
clustering_list = ['KM4', 'KM5', 'AHC4', 'AHC5', 'HDB_with_noise']

def agg_friedman(
    input_path : str,
    clustering_list : list,
):
    temp = np.empty((15, 1))
    for clus in clustering_list:
        data = pd.read_excel(input_path + clus + '/' + 'Friedman_Test.xlsx', engine = 'openpyxl')
        p_value = list(data['p_value'])
        for i in range(len(p_value)):
            p_value[i] = decimal_seperation(p_value[i])
        temp = np.hstack((temp, np.array(p_value).reshape(15, 1)))


    temp = pd.DataFrame(temp[:, 1:], columns = clustering_list)
    temp['Classifier_Metric'] = data.iloc[:, 0]
    temp = temp[['Classifier_Metric', 'KM4', 'KM5', 'AHC4', 'AHC5', 'HDB_with_noise']]
    temp.to_csv(input_path + '/Friedman_P-value.csv', index = False)

    return -1

agg_friedman('./Experimental_result/Final_result/', clustering_list)

-1

### **AGG P-value of Wilcoxon signed rank test**

In [96]:
clustering_list = ['KM4', 'KM5', 'AHC4', 'AHC5', 'HDB_with_noise'] #
classifier_list = ['DT', 'RF', 'NB', 'MLP', 'SVM']
compared_methods_list = ['None', 'ROS', 'SMOTE', 'B-SMOTE', 'ADASYN', 'SL-SMOTE', 'MWMOTE', 'KM-SMOTE', 'NRAS', 'SOMO', 'PF-SMOTE']


def agg_wilcoxon_test(
        input_path : str,
        clustering_list : list,
        compared_methods_list : list
):  
    temp1 = pd.DataFrame()
    temp1.to_excel(input_path + 'Wlicoxon_signed_rank_P-value.xlsx', sheet_name = 'Initialization', index = True)
    writer = pd.ExcelWriter(input_path + 'Wlicoxon_signed_rank_P-value.xlsx', mode = 'a')
    
    outcome = {i : np.empty(shape = (1, len(compared_methods_list))) for i in clustering_list}

    for clus in clustering_list:
        excel = pd.ExcelFile(input_path + clus + '/' + 'Wilcoxon_signed_rank_Test.xlsx')
        clf_list = excel.sheet_names[1:]
        idx = []
        for clf in clf_list:
            # Input experimental result metric by metric
            data = pd.read_excel(
                excel, 
                sheet_name = clf)
            p_value = np.array(data.iloc[:, 1:]).T.tolist()

            for m in range(data.shape[1] - 1):
                for i in range(len(p_value[0])):
                    p_value[m][i] = decimal_seperation(p_value[m][i])

            outcome[clus] = np.vstack((outcome[clus], np.array(p_value).reshape(3, len(compared_methods_list))))
            idx = idx + list(data.columns[1:])


        outcome[clus] = outcome[clus][1:, :]
        outcome[clus] = pd.DataFrame(outcome[clus], index = idx, columns = compared_methods_list)

        outcome[clus].to_excel(writer, sheet_name = clus, index = True, header = True)

    writer.close()
    return -1

agg_wilcoxon_test('./Experimental_result/Final_result/', clusterung_list = clusterung_list, compared_methods_list = compared_methods_list )


-1

### **Execution**

In [11]:

clf_list = ['DT', 'NB', 'RF','MLP', 'SVM']# ]#'SVM', ]
metrics_list = ['AUC', 'F1-score', 'G-mean']
clus = ['KM5', 'KM4', 'AHC5', 'AHC4', 'HDB_with_noise']#, ]#'HDB_rm_noise']
test_method = 'CIBO'
for c in clus:
    input_path = './Experimental_result/test_danger4/' + c + '/'


    Combining_experimetal_result(input_path = input_path, test_method = test_method, classifier_list = clf_list, metrics_list = metrics_list)
    Friedman_test(input_path = input_path, classifier_list = clf_list)
    Wilcoxon_signed_rank_test(input_path = input_path, classifier_list = clf_list, compared_method = 'CIBO')


c:\Users\KEE\anaconda3\envs\Dev-Resampling\lib\site-packages\scipy\stats\_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\KEE\anaconda3\envs\Dev-Resampling\lib\site-packages\scipy\stats\_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\KEE\anaconda3\envs\Dev-Resampling\lib\site-packages\scipy\stats\_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
c:\Users\KEE\anaconda3\envs\Dev-Resampling\lib\site-packages\scipy\stats\_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to no